1\. **Pandas DataFrame**

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every `x` BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
N=10000
data = pd.read_csv('data/data_000637.txt', nrows=N)

2\. Estimate the number of BX in a ORBIT (the value `x`).

*Hint*: check when the BX counter reaches the maximum value before being reset to 0.

In [3]:
x = data['BX_COUNTER'].max()
print("Estimated X value: ", x)
print(data.loc[0])

Estimated X value:  3563
HEAD                    1
FPGA                    0
TDC_CHANNEL           123
ORBIT_CNT      3869200167
BX_COUNTER           2374
TDC_MEAS               26
Name: 0, dtype: int64


3\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition, and convert the new column to a Time Series.

*Hint:* introduce an offset to the absolute time such that the start of the data acquisition (i.e. the first entry) is zero.

In [5]:
start_time = data.loc[0]['TDC_MEAS']*(25/30)+data.loc[0]['BX_COUNTER']*25+data.loc[0]['ORBIT_CNT']*x*25
abs_time = [0]
for i in range(1,N):
    curr_time = data.loc[i]['TDC_MEAS']*(25/30)+data.loc[i]['BX_COUNTER']*25+data.loc[i]['ORBIT_CNT']*x*25
    abs_time.append(curr_time-start_time)
data['ABS_TIME'] = abs_time
print(data.loc[6])

HEAD           1.000000e+00
FPGA           0.000000e+00
TDC_CHANNEL    6.100000e+01
ORBIT_CNT      3.869200e+09
BX_COUNTER     2.772000e+03
TDC_MEAS       1.400000e+01
ABS_TIME       9.940000e+03
Name: 6, dtype: float64


4\. Find out the duration of the data taking in hours, minutes and seconds, by using the features of the Time Series. Perform this check reading the whole dataset.

In [6]:
print("Total time to get", N, "misurations:", str(dt.timedelta(abs_time[N-1]/(10**9))))

Total time to get 10000 misurations: 0:12:46.157540


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [62]:
top_entries = data.groupby('TDC_CHANNEL').size().nlargest(3)
print("The 3 channels more noisy are:\n", top_entries)

The 3 channels more noisy are:
 TDC_CHANNEL
139    860
63     499
64     491
dtype: int64


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit).

7\. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

8\. Create two Series (one for each FPGA) that have the TDC channel as index, and the number of counts for the corresponding TDC channel as values.

9\. **Optional:** Create two histograms (one for each FPGA) that show the number of counts for each TDC channel.